<a href="https://colab.research.google.com/github/Saikrishnaraju0510/bigquery/blob/main/STREAM-TRIGGER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install apache-beam[gcp]

     |████████████████████████████████| 9.9 MB 4.4 MB/s 
     |████████████████████████████████| 151 kB 47.5 MB/s 
     |████████████████████████████████| 245 kB 61.5 MB/s 
     |████████████████████████████████| 2.3 MB 64.3 MB/s 
     |████████████████████████████████| 508 kB 62.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 435 kB 60.0 MB/s 
     |████████████████████████████████| 183 kB 65.6 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |████████████████████████████████| 172 kB 71.6 MB/s 
     |████████████████████████████████| 255 kB 65.6 MB/s 
     |████████████████████████████████| 144 kB 65.8 MB/s 
     |████████████████████████████████| 173 kB 63.0 MB/s 
     |████████████████████████████████| 180 kB 59.3 MB/s 
     |████████████████████████████████| 267 kB 65.0 MB/s 
     |████████████████████████████████| 111 kB 62.8 MB/s 
     |███████████████

In [1]:
!pip install google-apitools

In [9]:
from apache_beam.transforms.window import FixedWindows
from apache_beam.io.gcp.bigquery import WriteToBigQuery
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import argparse
import os
import time
from apache_beam.options.pipeline_options import SetupOptions
from datetime import date
from google.cloud import bigquery
from apache_beam import window
from apache_beam.transforms.trigger import AfterWatermark, AfterProcessingTime, AccumulationMode, AfterCount, Repeatedly

path1='key.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=path1
tab_id='airflow3.walletpayments1234'
import apache_beam as beam

parser = argparse.ArgumentParser()
# parser.add_argument('--my-arg')
args, beam_args = parser.parse_known_args()

# Create and set your Pipeline Options.
beam_options = PipelineOptions(
    beam_args,
    runner='DataflowRunner',
    temp_location='gs://sai_1999/',
    service_account_email='demo-py@modular-asset-332406.iam.gserviceaccount.com',
    region='us-central1',
    project='modular-asset-332406',
    stage_location='gs://sai_1999/stage',
    #save_main_session=True
    )
beam_options.view_as(StandardOptions).streaming=True
#beam_options.view_as(SetupOptions).save_main_session = True
client = bigquery.Client()

dataset_id = "modular-asset-332406.airflow3"

try:
	client.get_dataset(dataset_id)

except:
	dataset = bigquery.Dataset(dataset_id)  #

	dataset.location = "US"
	dataset.description = "dataset"

	dataset_ref = client.create_dataset(dataset)
 
def decoding(e):
  e=e.decode('utf-8')
  return e[:-2]
 
input_subscription='projects/modular-asset-332406/subscriptions/beam2-sub'
def to_json(fields):
    json_str = {"Invoice_ID":fields[0],
                 "Branch": fields[1],
                 "City": fields[2],
                 "Customer_type": fields[3],
                 "Gender": fields[4],
                 "Product_line": fields[5],
                 "Unit_price": fields[6],
                 "Quantity": fields[7],
                 "Tax_5": fields[8],
                 "Total": fields[9],
                 "Date": '256485',
                 "Time": fields[11],
                 "Payment": fields[12],
                 "gross_income": fields[15],
                 "Rating":fields[16]
                 }
    return json_str
schema_1='Invoice_ID:STRING,Branch:STRING,City:STRING,Customer_type:STRING,Gender:STRING,Product_line:STRING,Unit_price:STRING,Quantity:STRING,Tax_5:STRING,Total:STRING,Date:STRING,Time:STRING,Payment:STRING,gross_income:STRING,Rating:STRING'
with beam.Pipeline(options=beam_options) as pipeline:
  data_ingestion=(
    pipeline
    | beam.io.ReadFromPubSub(subscription=input_subscription,timestamp_attribute=None)
    | beam.Map(decoding)
    | beam.Map(lambda x:x.split(','))
    | beam.Map(to_json)
    | beam.WindowInto(beam.window.FixedWindows(5),trigger=AfterCount(3),accumulation_mode=AccumulationMode.DISCARDING)
    | beam.io.WriteToBigQuery(
                             tab_id,
                             schema=schema_1,
                             method=                                                                                
                     beam.io.WriteToBigQuery.Method.FILE_LOADS,
                             triggering_frequency=1, 
                             write_disposition=
                     beam.io.BigQueryDisposition.WRITE_APPEND,
                             create_disposition=
                     beam.io.BigQueryDisposition.CREATE_IF_NEEDED
        
    )
)

/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery.py:2102: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  self.table_reference.projectId = pcoll.pipeline.options.view_as(
/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery_file_loads.py:1112: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


KeyboardInterrupt: ignored